In [1]:
import numpy
import skimage.io

import ivvv

In [2]:
image = skimage.io.imread("AICS-10_5_5.ome.tif")

image = numpy.transpose(image, [0, 2, 3, 1])
image.shape

(65, 494, 306, 9)

In [4]:
ivvv.volshow(image, size=(512, 512), spacing=(1.0, 1.0, 4.0))

VolumeWidget(dimensions={'tile_width': 292, 'tile_height': 204, 'rows': 10, 'cols': 7, 'atlas_width': 2044, 'a…